# RoBERTa + Wikipedia RAG


In [ ]:
# Installing offline dependencies
!pip install -U --no-deps /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -U --no-deps /kaggle/input/datasets-214/datasets-2.14.5-py3-none-any.whl
!pip install -U --no-deps /kaggle/input/optimum-113/optimum-1.13.2-py3-none-any.whl
!pip install -U --no-deps /kaggle/input/transformers-432/transformers-4.32.1-py3-none-any.whl

In [ ]:
import gc
import logging
from time import time
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from threading import Condition
import ctypes
from functools import partial

import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# For RAG
import faiss
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_from_disk, Dataset

NUM_TITLES = 5
MAX_SEQ_LEN = 512
MODEL_PATH = "/kaggle/input/bge-small-faiss/"

# For LLM
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, AutoModel
from accelerate import init_empty_weights
from accelerate.utils.modeling import set_module_tensor_to_device
from safetensors.torch import load_file
from optimum.bettertransformer import BetterTransformer

N_BATCHES = 5
MAX_LENGTH = 4096
MAX_CONTEXT = 1200
# With NUM_TITLES = 5, the median lenght of a context if 1100 tokens (Q1: 900, Q3: 1400)

In [ ]:
# Function to clean RAM & vRAM
def clean_memory():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

# # Variable used to avoid running the notebook for 3 hours when submitting. Credit : CPMP
# df = pd.read_csv('/kaggle/input/bert-with-rag/train_context.csv')

# IS_TEST_SET = len(df) != 200

# Uncomment this to see results on the train set
df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv", index_col="id")
IS_TEST_SET = True
N_BATCHES = 1

In [ ]:
# New SentenceTransformer class similar to the one used in @Mgöksu notebook but relying on the transformers library only

class SentenceTransformer:
    def __init__(self, checkpoint, device="cuda:0"):
        self.device = device
        self.checkpoint = checkpoint
        self.model = AutoModel.from_pretrained(checkpoint).to(self.device).half()
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    def transform(self, batch):
        tokens = self.tokenizer(batch["text"], truncation=True, padding=True, return_tensors="pt", max_length=MAX_SEQ_LEN)
        return tokens.to(self.device)  

    def get_dataloader(self, sentences, batch_size=32):
        sentences = ["Represent this sentence for searching relevant passages: " + x for x in sentences]
        dataset = Dataset.from_dict({"text": sentences})
        dataset.set_transform(self.transform)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        return dataloader

    def encode(self, sentences, show_progress_bar=False, batch_size=32):
        dataloader = self.get_dataloader(sentences, batch_size=batch_size)
        pbar = tqdm(dataloader) if show_progress_bar else dataloader

        embeddings = []
        for batch in pbar:
            with torch.no_grad():
                e = self.model(**batch).pooler_output
                e = F.normalize(e, p=2, dim=1)
                embeddings.append(e.detach().cpu().numpy())
        embeddings = np.concatenate(embeddings, axis=0)
        return embeddings

In [ ]:
# if IS_TEST_SET:
    # Load embedding model
start = time()
print(f"Starting prompt embedding, t={time() - start :.1f}s")
model = SentenceTransformer(MODEL_PATH, device="cuda:0")

# Get embeddings of prompts
f = lambda row : " ".join([row["prompt"], row["A"], row["B"], row["C"], row["D"], row["E"]])
inputs = df.apply(f, axis=1).values # better results than prompt only
prompt_embeddings = model.encode(inputs, show_progress_bar=False)

# Search closest sentences in the wikipedia index 
print(f"Loading faiss index, t={time() - start :.1f}s")
faiss_index = faiss.read_index(MODEL_PATH + '/faiss.index')
# faiss_index = faiss.index_cpu_to_all_gpus(faiss_index) # causes OOM, and not that long on CPU

print(f"Starting text search, t={time() - start :.1f}s")
search_index = faiss_index.search(np.float32(prompt_embeddings), NUM_TITLES)[1]

print(f"Starting context extraction, t={time() - start :.1f}s")
dataset = load_from_disk("/kaggle/input/all-paraphs-parsed-expanded")
for i in range(len(df)):
    df.loc[i, "context"] = "-" + "\n-".join([dataset[int(j)]["text"] for j in search_index[i]])

# Free memory
faiss_index.reset()
del faiss_index, prompt_embeddings, model, dataset
clean_memory()
print(f"Context added, t={time() - start :.1f}s")

In [ ]:
from sklearn.model_selection import train_test_split

features = ['prompt', 'A', 'B', 'C', 'D', 'E', 'context']

y = df['answer']
X = df[features]
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [ ]:
from transformers import AutoTokenizer
# comment the line below if you are finetuning RoBERTa
model_dir = '/kaggle/input/finetuned-roberta-base-rag-context/finetuned_roberta-base-rag-context/checkpoint-150' #you need to change this line based on the path to your finetuned model
# comment the line below if you are using a finetuned RoBERTa
# model_dir = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [ ]:
options = 'ABCDE'
indices = list(range(5))

option_index = {option: index for option, index in zip(options, indices)}
index_option = {index: option for option, index in zip(options, indices)}
train_dataset = Dataset.from_pandas(df)

def preprocess(examples):
    # adding context to our prompt
    text = examples['context'] + examples['prompt']
    first_sentences = [text] * 5
    second_sentences = []
    for option in options:
        second_sentences.append(examples[option])
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    tokenized_examples['label'] = option_index[examples['answer']]
    return tokenized_examples

tokenized_train_ds = train_dataset.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer', 'context'])

In [ ]:
# Following datacollator (adapted from https://huggingface.co/docs/transformers/tasks/multiple_choice)
# will dynamically pad our questions at batch-time so we don't have to make every question the length
# of our longest question.
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
model = AutoModelForMultipleChoice.from_pretrained(model_dir)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(p):
    # p.predictions are the predictions, p.label_ids are the true labels
    preds = p.predictions.argmax(-1)
    
    accuracy = accuracy_score(p.label_ids, preds)
    f1 = f1_score(p.label_ids, preds, average='weighted')  # Use 'binary' for binary classification
    return {
        'accuracy': accuracy,
        'f1': f1
    }

In [ ]:
from transformers import TrainerCallback

# this class allows us to save the path to the best performing model
class BestModelTracker(TrainerCallback):
    def __init__(self):
        self.best_loss = float('inf')
        self.best_epoch = 0
        self.best_model_path = None

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        # Check if the current eval loss is lower than the best loss
        if metrics and metrics.get("eval_loss", float('inf')) < self.best_loss:
            self.best_loss = metrics["eval_loss"]
            self.best_epoch = state.epoch
            self.best_model_path = f'{args.output_dir}/checkpoint-{state.global_step}'
            print(f"New best model found at epoch {self.best_epoch} with eval loss: {self.best_loss}")

best_model_tracker = BestModelTracker()

model_dir = 'finetuned_roberta-base-rag-context'
training_args = TrainingArguments(
    output_dir=f'{model_dir}',
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="epoch",
        metric_for_best_model='eval_loss',  # Choose the metric to determine the best model
        greater_is_better=False,  # Set to True if higher metric score is better
        load_best_model_at_end=True,
        learning_rate=5e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=15,
        weight_decay=0.01,
        report_to='none',
        save_total_limit=1
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_train_ds,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
    callbacks=[best_model_tracker]
)

In [ ]:
# uncomment if your notebook has access to Internet and you want to measure CPU/GPU usage
# !pip install psutil GPUtil

In [ ]:
# uncomment if your notebook has access to Internet and you want to measure CPU/GPU usage
# import threading
# import psutil
# import GPUtil
# import time
# import matplotlib.pyplot as plt

# # Function to monitor CPU and GPU usage
# def monitor_resources(stop_event, interval=1):
#     cpu_usage = []
#     gpu_usage = []
#     timestamps = []

#     while not stop_event.is_set():
#         timestamps.append(time.time())
#         cpu_usage.append(psutil.cpu_percent(interval=None))

#         gpus = GPUtil.getGPUs()
#         gpu_usage.append(gpus[0].load * 100 if gpus else 0)  # Assumes one GPU

#         time.sleep(interval)

#     plt.figure(figsize=(10, 6))
#     plt.plot(timestamps, cpu_usage, label='CPU Usage (%)')
#     plt.plot(timestamps, gpu_usage, label='GPU Usage (%)')
#     plt.xlabel('Time (s)')
#     plt.ylabel('Usage (%)')
#     plt.title('CPU and GPU Usage Over Time')
#     plt.legend()
#     plt.grid(True)
#     plt.show()
    
#     print(timestamps)
#     print(cpu_usage)
#     print(gpu_usage)

In [ ]:
# all lines except trainer.train() are for measuring CPU/GPU usage

# # Monitoring setup
# stop_event = threading.Event()
# monitor_thread = threading.Thread(target=monitor_resources, args=(stop_event,))

# # Start monitoring
# monitor_thread.start()

# # Start training
# trainer.train()

# # Stop monitoring
# stop_event.set()
# monitor_thread.join()

# # At this point, the monitoring thread has completed
# print("Training and monitoring completed.")

In [ ]:
#load the best model
# model = AutoModelForMultipleChoice.from_pretrained(best_model_tracker.best_model_path)
# training_args = TrainingArguments(
#         output_dir=f'{model_dir}',
#         evaluation_strategy="epoch",
#         logging_strategy="epoch",
#         save_strategy="epoch",
#         metric_for_best_model='eval_loss',  # Choose the metric to determine the best model
#         greater_is_better=False,  # Set to True if higher metric score is better
#         load_best_model_at_end=True,
#         learning_rate=5e-5,
#         per_device_train_batch_size=4,
#         per_device_eval_batch_size=4,
#         num_train_epochs=15,
#         weight_decay=0.01,
#         report_to='none',
#         save_total_limit=1
#     )

#     # Initialize the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train_ds,
#     eval_dataset=tokenized_train_ds,
#     tokenizer=tokenizer,
#     data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
#     compute_metrics=compute_metrics,
#     callbacks=[best_model_tracker]
# )

In [ ]:
#uncomment if you want to make predictions on the training set and measure CPU/GPU usage

# # Monitoring setup
# stop_event = threading.Event()
# monitor_thread = threading.Thread(target=monitor_resources, args=(stop_event,))

# # Start monitoring
# monitor_thread.start()

# # Start training
# predictions = trainer.predict(tokenized_train_ds)

# # Stop monitoring
# stop_event.set()
# monitor_thread.join()

# # At this point, the monitoring thread has completed
# print("Prediction inference and monitoring completed.")

In [ ]:
#function to generate three letter options

import numpy as np
def predictions_to_map_output(predictions):
    sorted_answer_indices = np.argsort(-predictions)
    top_answer_indices = sorted_answer_indices[:,:3] # Get the first three answers in each row
    top_answers = np.vectorize(index_option.get)(top_answer_indices)
    return np.apply_along_axis(lambda row: ' '.join(row), 1, top_answers)

In [ ]:
#estimate MAP@3
def map_at_3(predictions, true_answers):
    # Convert predictions to top 3 answers
    top_3_predictions = predictions_to_map_output(predictions.predictions)

    # Calculate average precision for each instance
    average_precisions = []
    for i in range(len(true_answers)):
        true_answer = true_answers[i]
        true_answer = options[true_answer]
        predicted_answers = top_3_predictions[i].split(" ")

        if true_answer in predicted_answers:
            index_of_true_answer = predicted_answers.index(true_answer)
            precision_at_index = 1 / (index_of_true_answer + 1)
            average_precisions.append(precision_at_index)
        else:
            average_precisions.append(0)

    # Calculate mean average precision at 3
    map_3 = np.mean(average_precisions)
    return map_3

In [ ]:
#uncomment if you want to get MAP@3 score for the training set
# true_answers = tokenized_train_ds['label']
# map_3_score = map_at_3(predictions, true_answers)
# print(f"MAP@3 Score: {map_3_score}")

In [ ]:
test_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')

In [ ]:
# get context for a test dataset
import time
start = time.time()
print(f"Starting prompt embedding, t={time.time() - start :.1f}s")
model = SentenceTransformer(MODEL_PATH, device="cuda:0")

# Get embeddings of prompts
f = lambda row : " ".join([row["prompt"], row["A"], row["B"], row["C"], row["D"], row["E"]])
inputs = test_df.apply(f, axis=1).values # better results than prompt only
prompt_embeddings = model.encode(inputs, show_progress_bar=False)

# Search closest sentences in the wikipedia index 
print(f"Loading faiss index, t={time.time() - start :.1f}s")
faiss_index = faiss.read_index(MODEL_PATH + '/faiss.index')
# faiss_index = faiss.index_cpu_to_all_gpus(faiss_index) # causes OOM, and not that long on CPU

print(f"Starting text search, t={time.time() - start :.1f}s")
search_index = faiss_index.search(np.float32(prompt_embeddings), NUM_TITLES)[1]

print(f"Starting context extraction, t={time.time() - start :.1f}s")
dataset = load_from_disk("/kaggle/input/all-paraphs-parsed-expanded")
for i in range(len(test_df)):
    test_df.loc[i, "context"] = "-" + "\n-".join([dataset[int(j)]["text"] for j in search_index[i]])

# Free memory
faiss_index.reset()
del faiss_index, prompt_embeddings, model, dataset
clean_memory()
print(f"Context added, t={time.time() - start :.1f}s")

In [ ]:
test_df['answer'] = 'A'

# Other than that we'll preprocess it in the same way we preprocessed test.csv
test_ds = Dataset.from_pandas(test_df)
tokenized_test_ds = test_ds.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])

In [ ]:
# uncomment this if you want to measure CPU/GPU usage of making predictions on the test set

# stop_event = threading.Event()
# monitor_thread = threading.Thread(target=monitor_resources, args=(stop_event,))

# # Start monitoring
# monitor_thread.start()

# # Start training
# test_predictions = trainer.predict(tokenized_test_ds)
# # Stop monitoring
# stop_event.set()
# monitor_thread.join()

# # At this point, the monitoring thread has completed
# print("Test prediction inference and monitoring completed.")

# run only this to make predicitons on the test set
test_predictions = trainer.predict(tokenized_test_ds)

In [ ]:
# create a submission file
submission_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/sample_submission.csv')
test_df['prediction'] = predictions_to_map_output(test_predictions.predictions)
submission_df['prediction'] = test_df['prediction']
submission_df.head()

In [ ]:
submission_df.to_csv('submission.csv', index=False)